In [1]:
import numpy as np
import glob
import emcee as mc
import h5py
from gaussianLikelihoods import logp_powerLawPeak
from tqdm.auto import tqdm

In [2]:
#AI-generated new version
def run_mcmc_estimation(sample_dict_path, injection_dict_path, output_path, 
                       n_walkers=32, n_steps=20000, n_threads=16):
    """
    Run MCMC estimation using sample and injection data.
    
    Parameters:
    -----------
    sample_dict_path : str
        Path to the sample dictionary pickle file
    injection_dict_path : str
        Path to the injection dictionary pickle file
    output_path : str
        Path where the MCMC chain results should be saved
    n_walkers : int, optional
        Number of walkers for MCMC (default: 32)
    n_steps : int, optional
        Number of MCMC steps (default: 20000)
    n_threads : int, optional
        Number of threads to use for parallel processing (default: 16)
    
    Returns:
    --------
    str
        Path to the saved MCMC chain file
    """
    # Prior bounds dictionary
    prior_dict = {
        'lmbda': (-5, 4),
        'mMax': (60, 100),
        'm0': (20, 100),
        'sigM': (1, 10),
        'fPeak': (0, 1),
        'bq': (-2, 10),
        'sig_kappa': 6.,
        'mu_eff': (-1, 1),
        'sigma_eff': (0.05, 1.),
        'mu_p': (0.05, 1),
        'sigma_p': (0.05, 1.),
        'rho': (-0.75, 0.75),
        'mMin': 5.
    }
    
    # Load input data
    sample_dict = np.load(sample_dict_path, allow_pickle=True)
    if 'S190814bv' in sample_dict:
        sample_dict.pop('S190814bv')
    
    injection_dict = np.load(injection_dict_path, allow_pickle=True)
    
    # Parameter space dimension
    dim = 12
    
    # Initialize chain or continue from existing one
    old_chains = np.sort(glob.glob(f"{output_path}_r??.npy"))
    
    if len(old_chains) == 0:
        # Start new chain
        run_version = 0
        initial_walkers = initialize_new_walkers(n_walkers)
    else:
        # Continue existing chain
        old_chain = np.load(old_chains[-1])
        run_version = int(old_chains[-1][-6:-4]) + 1
        
        # Remove trailing zeros from incomplete runs
        good_inds = np.where(old_chain[0,:,0] != 0.0)[0]
        old_chain = old_chain[:,good_inds,:]
        
        # Use final positions as new initial positions
        initial_walkers = old_chain[:,-1,:]
    
    # Initialize and run sampler
    sampler = mc.EnsembleSampler(
        n_walkers, 
        dim, 
        logp_powerLawPeak,
        args=[sample_dict, injection_dict, prior_dict],
        threads=n_threads
    )
    
    # Run MCMC with periodic saving
    #for i, result in enumerate(sampler.sample(initial_walkers, iterations=n_steps)):
    #    if i % 10 == 0:
    #        save_path = f"{output_path}_r{run_version:02d}.npy"
    #        np.save(save_path, sampler.chain)
     
        

    # Run MCMC with progress bar and periodic saving
    print(f"\nRunning MCMC with {n_walkers} walkers for {n_steps} steps...")
    save_interval = 10  # Save every 10 steps
    
    # Initialize progress bar
    with tqdm(total=n_steps, desc="MCMC Progress") as pbar:
        for i, result in enumerate(sampler.sample(initial_walkers, iterations=n_steps)):
            if i % save_interval == 0:
                save_path = f"{output_path}_r{run_version:02d}.npy"
                np.save(save_path, sampler.chain)
            
            # Update progress bar
            pbar.update(1)
            
            # Add acceptance fraction to progress bar description every 100 steps
            if i % 100 == 0:
                acc_frac = np.mean(sampler.acceptance_fraction)
                pbar.set_description(f"MCMC Progress (acceptance: {acc_frac:.3f})")
   


    # Final save
    save_path = f"{output_path}_r{run_version:02d}.npy"
    np.save(save_path, sampler.chain)
    
    return save_path

def initialize_new_walkers(n_walkers):
    """
    Initialize walker positions for a new MCMC chain.
    
    Parameters:
    -----------
    n_walkers : int
        Number of walkers to initialize
    
    Returns:
    --------
    numpy.ndarray
        Initial walker positions
    """
    initial_lmbdas = np.random.random(n_walkers) * (-2.)
    initial_mMaxs = np.random.random(n_walkers) * 20. + 80.
    initial_m0s = np.random.random(n_walkers) * 10. + 30
    initial_sigMs = np.random.random(n_walkers) * 4 + 1.
    initial_fs = np.random.random(n_walkers)
    initial_bqs = np.random.random(n_walkers) * 2.
    initial_ks = np.random.normal(size=n_walkers, loc=0, scale=1) + 2.
    initial_mu_effs = np.random.random(n_walkers) * 0.05
    initial_sigma_effs = np.random.random(n_walkers) * 0.2 + 0.05
    initial_mu_ps = np.random.random(n_walkers) * 0.5 + 0.05
    initial_sigma_ps = np.random.random(n_walkers) * 0.2 + 0.05
    initial_rhos = np.random.random(n_walkers) * 0.4 - 0.2
    
    return np.transpose([
        initial_lmbdas, initial_mMaxs, initial_m0s, initial_sigMs,
        initial_fs, initial_bqs, initial_ks, initial_mu_effs,
        initial_sigma_effs, initial_mu_ps, initial_sigma_ps, initial_rhos
    ])

In [3]:
run_mcmc_estimation(sample_dict_path="../input/sampleDict_FAR_1_in_1_yr_rerun_K.pickle",
                    injection_dict_path="./injectionDict_rerun_directMixture_FAR_1_in_1_K.pickle",
                    output_path="../results/KDE_result",
                    n_walkers=32,
                    n_steps=20000,
                    n_threads=16)


Running MCMC with 32 walkers for 20000 steps...


MCMC Progress:   0%|          | 0/20000 [00:00<?, ?it/s]

GW170729 8.906575868447371
GW170729 9.343714263776526
S190521g 1.0000000001834606
GW170729 6.3178708760067
GW170729 9.32154326968652
S190521g 7.9717612825493696
GW170729 8.453125742521573
GW170729 9.953266240905382
GW170729 6.215330612345874
GW170729 4.856613390534234
S190517h 3.1433796405779737
GW170729 9.5217195119692
GW170729 7.603641794865802
S190517h 6.218251756458096
GW170729 8.718632679423832
GW170729 6.9104209475089515
GW170729 9.68889932352642
GW170729 9.195809376336644
S190517h 7.842799837114267
GW170729 9.410555420270615
GW170729 9.253455541637562
GW170729 7.316527597360259
S190521g 1.8154025741867597
GW170729 7.017537437218423
GW170729 8.913696484219932
GW170729 8.816037457555575
GW170729 9.996500437971827
GW170729 8.389854305172177
GW170729 8.906046652943045
GW170729 8.87932484254431
GW170729 8.410093431510742
GW170729 9.615460355613829
GW170729 9.631249681538586
GW170729 9.966489600407636
GW170729 8.851732192801355
GW170729 5.117445925204811
GW170729 9.683212203656046
GW1

GW170729 6.275356369621293
S190517h 8.839252078196743
S190521g 5.315043979564345
GW170729 8.203532824399772
S190521g 1.0000012497421302
GW170729 9.023969199483709
GW170729 9.864736178602271
S190521g 4.627297866832083
GW170729 9.173380081219683
GW170729 9.970237804862284
S190521g 3.198102940069252
GW170729 5.1778660935740835
GW170729 8.59730480020727
GW170729 6.723580035946585
GW170729 5.152712854473706
S190521g 8.643204893941055
GW170729 7.585937506104689
GW170729 9.751467325442023
S190521g 5.253756168094482
GW170729 6.642838339830948
S190521g 2.5655737294663647
GW170729 9.36736234562784
GW170729 7.711799930520967
S190521g 7.055993940618269
GW170729 8.576491128193693
S190521g 8.416003054271144
GW170729 6.673389303012237
GW170729 8.433101137402424
GW170729 7.992718682959177
S190521g 3.839073636499172
GW170729 6.947190816466374
GW170729 6.284010577952651
S190521g 3.427987353331023
GW170729 6.5865900354489035
S190521g 4.896122744488264
GW170729 9.940854891986346
GW170729 3.845793453600840

S190521g 1.0141720652300716
S190521g 2.906198692757005
GW170729 9.14363773874961
GW170729 8.695551213431305
S190521g 1.0023706365774163
GW170729 9.0721113486356
GW170729 9.689039348554362
GW170729 7.855750586714414
GW170729 6.4269012122609945
S190521g 6.826517865484254
S190521g 7.197996654009003
S190521g 3.0782923563879327
GW170729 7.522559355114177
S190521g 1.156928018163283
S190521g 8.745368089660573
S190517h 9.129763665253535
GW170729 6.782621488419213
S190521g 2.5207071696528036
S190519bj 8.711658255696822
GW170729 5.734521877861759
S190521g 5.4935070301962945
GW170729 8.689347751934246
S190521g 1.1346645586282387
S190517h 8.687577280963156
GW170729 8.708711995095417
GW170729 8.705743186254455
S190519bj 9.845293493592933
GW170729 9.881276740936311
GW170729 6.179456025382759
S190521g 3.2173688420619304
GW170729 6.751824434778415
S190521g 7.554335810914178
S190521g 4.135275792595677
S190517h 5.222018712819565
GW170729 8.174433260784634
GW170729 9.31387051444759
S190620e 9.36457989976

...

S190521g 2.8951493941887136
S190521g 1.7537136753980618
GW170729 8.259281588600928
GW170729 6.713401257078034
GW170729 9.085551380048422
GW170729 8.83519296629601
GW170729 9.867382421831678
S190517h 9.947856062599271
GW170729 9.453481504739711
GW170729 9.680896486830006
GW170729 9.410184981192357
S190521g 1.0000028636828302
S190521g 5.830257425990745
S190521g 1.4363328835480993
GW170729 7.879764103415847
S190517h 8.256085098334008
GW170729 6.9690347921163385
GW170729 9.840123153066138
S190517h 5.375884469107399
GW170729 9.707770016442899
GW170729 9.343034041584287
S190521g 2.898402775420379
GW170729 7.363309697799122
GW170729 8.455489972097933
GW170729 6.824745968526269
GW170729 7.737123068359006
GW170729 9.42758821417614
S190517h 6.714574897637534
GW170729 7.020388590077808
S190521g 1.4301080886014086
GW170729 6.5686713915247275
GW170729 4.536235139052942
GW170729 9.43133383358086
GW170729 9.276279426192223
GW170729 5.023475397186423
S190521g 1.0117047717774952
GW170729 8.477839612886

GW170729 7.384438757951508
GW170729 8.464995589038473
GW170729 8.953394560842815
GW170729 5.047102396638196
GW170729 9.619127817737583
GW170729 8.633562892164566
GW170729 9.687372535786569
GW170729 8.653277391237474
S190620e 9.204255478648168
GW170729 8.348628210905005
GW170729 8.075763814464647
Insufficient mock detections: [-3.45156802e+00  9.37223456e+01  3.60422833e+01  1.19873739e+00
  6.18606309e-02  1.39188959e+00  3.89868362e+00  1.01154143e-01
  1.16390596e-01  5.89473070e-02  7.68201344e-02  2.62400614e-01]
S190521g 6.8147899039067825
GW170729 7.356626824018557
S190517h 8.966577598073128
GW170729 9.286100609229681
GW170729 9.34783814779767
S190517h 3.746302238592288
S190521g 1.0693769668544297
GW170729 5.309292760555288
Insufficient mock detections: [-4.24509602e+00  8.31983714e+01  3.22709210e+01  6.59743018e+00
  1.38060610e-02  2.73007986e+00  3.09060580e+00  7.37196238e-02
  9.39764241e-02  5.73569125e-02  5.00624271e-02  3.49151794e-02]
GW170729 9.0416990283238
GW170729 

GW170729 8.926972627787416
GW170729 7.1644514768283685
GW170729 9.466997471527232
S190521g 9.241806191161398
S190521g 6.680008026808252
S190521g 3.1447906459882478
S190521g 7.815010870133367
S190521g 8.403757250827775
GW170729 9.526322399924595
S190521g 8.662074066669847
S190521g 8.002846339481348
GW170729 7.606007640898916
GW170729 9.022034660944538
GW170729 9.286491980207554
GW170729 6.01983204014322
GW170729 7.544690115164541
GW170729 9.503348444564741
GW170729 7.768962174656989
GW170729 9.807203823190772
GW170729 9.064398360965566
GW170729 9.284291196278168
GW170729 6.661995621157027
Insufficient mock detections: [-3.76546688e+00  6.92170361e+01  3.39536916e+01  1.08148115e+00
  3.70313200e-02  9.15124462e-01  7.27765092e+00  1.14244364e-01
  1.05369938e-01  6.63694670e-02  1.10848453e-01 -2.22398323e-01]
S190521g 5.438728375647247
S190521g 2.5902374143535494
GW170729 8.871327504604341
S190521g 7.431903982773293
S190521g 5.202628111405543
GW170729 9.333733191216096
GW170729 7.34083

S190519bj 4.7679054163549175
GW170729 8.700229168149281
GW170729 7.4242087367669285
S190519bj 3.4730390390533556
GW170729 8.54777888706067
GW170729 9.303314633957275
GW170729 9.20021780429977
S190519bj 6.8209488275981
S190521g 4.928057290855543
Insufficient mock detections: [-4.36748065e+00  7.73397561e+01  3.65510546e+01  1.20207692e+00
  3.46077467e-02  5.17564453e+00  6.47463136e+00  4.49250641e-02
  7.40845218e-02  5.73488861e-02  1.00837784e-01  8.94727815e-02]
GW170729 8.669511500886992
GW170729 9.98806580227413
GW170729 7.517002918305643
GW170729 5.983886582015819
GW170729 9.921633829346078
GW170729 9.455011183186746
GW170729 9.601481753914618
S190521g 5.111534343709036
Insufficient mock detections: [-4.67650385e+00  7.14793273e+01  3.58623697e+01  2.94738446e+00
  2.38340881e-02  3.15098546e+00  7.11943409e+00  6.19941810e-02
  7.43698859e-02  8.41674139e-02  6.58424133e-02  4.11484074e-01]
S190519bj 2.5133056724675105
GW170729 9.947967608331822
GW170729 4.861515163168397
GW170

'../results/KDE_result_r01.npy'

In [4]:
run_mcmc_estimation(sample_dict_path="../input/sampleDict_FAR_1_in_1_yr_rerun_A.pickle",
                    injection_dict_path="./injectionDict_rerun_directMixture_FAR_1_in_1_A.pickle",
                    output_path="../results/Analytical_result",
                    n_walkers=32,
                    n_steps=20000,
                    n_threads=16)


Running MCMC with 32 walkers for 20000 steps...


MCMC Progress:   0%|          | 0/20000 [00:00<?, ?it/s]

GW170729 9.034266331758033
S190620e 9.597283913690903
S190521g 1.6953053929926791
S190620e 8.728690412203393
S190521g 7.672420633406269
S190521g 9.119913751804134
GW151226 8.107775413119048
S190521g 9.565013276536098
S190521g 6.3759074534727205
GW170729 8.005034495150959
S190521g 6.412531784009029
S190521g 8.154157178706475
S190521g 5.746942582488325
S190521g 9.719206111973426
S190620e 3.9725771903597527
S190521g 8.826534073058431
S190521g 9.472783126183856
S190521g 3.232140369362836
S190521g 6.456291805319289
S190521g 1.1399451114457546
S190620e 9.862526293138952
S190620e 9.584943752848151
GW151226 9.967045971935416
GW170729 8.568370072723562
S190620e 9.289630917898492
S190521g 7.628563617844869
S190521g 8.142765622433895
S190521g 9.777976715806135
S190521g 3.058568903701246
S190521g 5.719812657052062
S190521g 8.76508349421958
S190521g 8.88346620351829
S190521g 6.15225641708014
S190521g 1.0061989575795642
S190620e 9.143284378451055
S190521g 6.815141314418304
S190521g 9.106748435999913

S190521g 5.847689692698481
S190521g 5.850927341903727
S190521g 7.160930207010386
S190521g 6.7533928858801335
S190521g 9.336362954877309
S190620e 7.875234574948158
S190620e 5.519407132003287
S190521g 6.266506722355226
S190620e 8.825219445091886
S190620e 7.276588270323996
S191230an 9.988829083602608
S190620e 7.9775473228535585
S190521g 8.47650807582947
GW170729 7.517429227243133
S190521g 5.753098154666902
S190521g 6.943898229419902
S190521g 5.731673724164893
S190620e 8.973673366833504
S190521g 9.947614623354355
S190521g 8.215707568816052
S190620e 9.551215225260359
S190521g 9.426384418465632
S190517h 8.093992253429935
S190521g 9.198055810955557
S190521g 7.617082441170714
S190620e 8.55558782995791
GW170729 9.527247272588676
S191230an 8.812577904986606
S190620e 8.192002971322978
GW170729 9.9632008891883
S190521g 6.913018323660419
S191230an 9.817396774290058
S190521g 5.631007565548418
S190521g 4.454039282511306
S190521g 9.797278662659915
S190521g 7.467256563063337
S190521g 4.205890388223312


S190620e 9.573713655296368
S190521g 4.634914956609227
S190521g 5.422258745293944
S191230an 8.299928995676348
S190521g 8.997826728219394
S190620e 9.636591310448008
S190620e 9.291954765455122
S190521g 9.737950777326564
S190521g 5.722058735351342
GW170729 8.97427322034916
S191230an 9.798028190411053
S190620e 6.570821470145299
GW170729 8.173316058419394
S190620e 9.12692331153485
S190521g 8.186587020035695
S190620e 5.893255847102216
S190521g 6.708562824992497
S190521g 7.299737806542377
S190521g 9.091703915223153
S190620e 9.904186296155899
GW170729 9.13784224476983
S190521g 5.991676651124393
GW170729 9.614361606082577
S190521g 6.587844673213628
GW170729 5.719506321269491
S190521g 6.834006319702156
GW170729 9.598777547595484
S190521g 6.486953213633022
GW170729 9.782996332617092
GW170729 7.560218798959863
S190521g 7.005773908900253
S190521g 5.811221206267426
GW170729 8.314157312512883
S190521g 4.343749106404021
S190620e 4.737461820901723
S190521g 8.182503175921621
S190521g 4.36134321213983
S19

...

S190521g 5.828923125347722
S190620e 8.721256683166317
S190521g 6.891453130995035
S190521g 6.893631713795845
S190521g 8.699336589109725
S190620e 7.835621943011272
S190521g 8.922827382507114
S190521g 5.871700783171613
S190521g 6.370015061811769
S190521g 7.335565551569884
S190521g 5.73504989224
S190620e 7.703700430681771
S190521g 1.3950881188591409
S190521g 9.209830088681835
S190521g 4.9743153936007
S190521g 9.870939469834841
S190521g 6.772121251583755
S190521g 9.670255499415523
S190521g 7.316083394708758
S190521g 1.0
S190521g 8.499370340400995
GW170729 7.525677592218842
S190521g 1.781761656895732
S190521g 9.261588960963907
S190521g 3.3836033324029984
S190521g 9.150354820405761
S190521g 6.613392105838723
S190521g 8.150877254079333
S190521g 9.156168669183423
S190521g 2.0745865456410044
S190521g 7.205299037404609
S190620e 8.516593174108177
GW170729 9.250973984748427
S190521g 8.292245430862105
S190521g 2.6040238966852707
S190521g 6.5742133646263
S190521g 6.689332843571362
S190521g 6.27552336

GW151226 9.627558070707778
S190521g 6.0846971817840725
S190521g 8.096203640605527
S190521g 5.45163510703309
GW151226 9.192352757162036
S190620e 8.10078790591623
S190521g 4.679436685206882
S190620e 8.3397270053489
S190521g 9.757317229994749
S190521g 2.505210199415267
S190521g 8.285356466756735
GW170729 9.786932392930327
S190521g 6.058799869860715
S190521g 7.29697076506405
S190521g 5.057128720098938
S190521g 7.767422198703395
S190521g 9.292575761684123
GW151226 7.630100391198123
S190521g 4.067396771520739
S190521g 2.1460215323770284
GW151226 6.026046460053663
S190521g 6.274832144624828
S190521g 9.904845989263736
S190521g 3.751024194806075
S190620e 8.688517904035395
S190521g 9.74083184453913
S190521g 9.197143625869124
GW151226 6.983804848908601
S190521g 3.6905754645677744
S190620e 8.505782035792755
S190521g 6.486682622623979
S190620e 9.900744701295675
S190521g 5.10219302557904
S190521g 4.32651841375005
S190620e 7.255443107451859
GW170729 5.863334061813614
S190521g 8.074100989336907
S19052

S190521g 8.05155633424539
S190521g 3.1948567347729515
S190620e 9.297787440178418
S190521g 4.63231411559252
S190620e 8.443231070386426
S190521g 5.5283518481629494
GW170729 4.510919002173436
S190521g 8.817431547801137
S190521g 8.004480646634283
S190521g 8.459761404086516
S191230an 8.810697640597287
S190521g 2.9551217377574543
S191230an 7.063591343424477
S190521g 6.850911050330528
S190620e 8.387732912605635
S190521g 4.713224238613737
GW151226 9.100005338116363
S190521g 8.917424336146317
S190521g 7.85373738653426
S190521g 6.988601125941044
S191230an 7.870902691255302
S190521g 8.730648097323215
S190521g 7.0933890455521125
GW170729 4.54800572916133
S190521g 9.731992336534665
GW170729 8.575136471295338
S191230an 7.646741210835205
S190521g 6.44717723338383
S190620e 7.85152228403975
S190521g 7.658958550707233
S190521g 7.326174849644934
S190620e 9.462986632609846
S190521g 6.540950226869763
S190521g 5.303473841836984
S190521g 7.292883766021067
S190521g 8.697530652979003
S190521g 4.039346694328373

S190521g 7.589681448214254
S190521g 9.426248624353576
S190521g 4.70215752884591
S190620e 8.228552071460065
GW170729 5.0744625208071
S190521g 8.938088842631085
S191230an 9.441592401979976
S190521g 7.803073278162622
S190521g 9.619068322814828
S190521g 2.484143588277057
S190521g 1.4622718587996308
S190521g 6.165426390872115
S190521g 9.956421880731023
S190521g 3.256271084233207
S190521g 2.280589825329994
S190521g 5.475878860746384
S190521g 1.9718822469108588
S190521g 5.314019700986054
S190620e 8.858746918186336
S190517h 3.275414722790831
S190521g 1.9201036935551339
S190521g 7.775692586954633
S191230an 6.748052203620606
GW170729 9.647907489814159
GW170729 4.322193439023207
S190620e 9.33411796739566
S190521g 5.982990949364432
S190521g 9.734184549679812
S190521g 1.000445020102805
S190517h 6.387625927778324
GW170729 6.872233795164379
S190521g 9.850161227150197
S190521g 7.039241951405819
S191230an 9.544450625082643
GW170729 7.44707813575391
S190521g 9.151099318919936
S190521g 1.6801840298248611

'../results/Analytical_result_r01.npy'